In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str # 사용자 질문
    answer: str # 세율
    tax_base_equation: str # 과세표준 계산 수식 
    tax_deduction: str # 공제액 
    market_ratio: str # 공정시장가액비율
    tax_base: str # 과세표준 계산
    
graph_builder = StateGraph(AgentState)

In [16]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('./real_estate_tax.txt', encoding='utf-8')
data = loader.load()
len(data)

1

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

texts = text_splitter.split_documents(data)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal
from langchain_openai import ChatOpenAI

# 라우팅 결과 구조 정의
class Route(BaseModel):
    target: Literal['income_tax', 'real_estate_tax', 'basic_llm', 'web_search'] = Field(
        description="사용자 질문을 처리할 적절한 경로"
    )

router_system_prompt = """
당신은 세금 전문가 질문 라우터입니다.
사용자의 질문을 다음 중 어디로 보낼지 판단하세요:
- 'income_tax': 소득세 관련 질문
- 'real_estate_tax': 부동산세 관련 질문
- 'basic_llm': 단순 질의 (벡터 검색 필요 없음)
- 'web_search': 최신 정보 필요할 때
"""

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", router_system_prompt),
        ("human", "{query}")
    ]
)

router_llm = ChatOpenAI(model="gpt-4o-mini")
structured_router_llm = router_llm.with_structured_output(Route)

# 라우터 함수
def router(state: AgentState) -> Literal['income_tax', 'real_estate_tax', 'basic_llm', 'web_search']:
    query = state['query']
    route_chain = router_prompt | structured_router_llm
    route = route_chain.invoke({"query": query})
    print("선택된 경로:", route.target)
    return route.target


In [ ]:
from langgraph.graph import START, END

def income_tax_agent(state: AgentState) -> AgentState:
    context = income_tax_retriever.invoke(state["query"])
    chain = (income_tax_prompt | llm | StrOutputParser())
    state["answer"] = chain.invoke({"context": context, "query": state["query"]})
    return state



def real_estate_tax_agent(state: AgentState) -> AgentState:
    """
    부동산세 관련 질문을 처리하는 노드입니다.
    입력된 state의 query를 바탕으로 벡터DB와 LLM을 활용해 부동산세 정보를 반환합니다.
    
    Args:
        state (AgentState): 현재 에이전트의 state 객체입니다.
    
    Returns:
        AgentState: 'answer' 키에 부동산세 결과가 포함된 state를 반환합니다.
    """
    state['answer'] = "부동산세 계산 결과입니다."
    return state


def basic_llm(state: AgentState) -> AgentState:
    """
    단순한 일반 질문을 처리하는 노드입니다.
    벡터DB나 웹 검색을 거치지 않고, LLM의 직답으로 처리합니다.
    
    Args:
        state (AgentState): 현재 에이전트의 state 객체입니다.
    
    Returns:
        AgentState: 'answer' 키에 단순 질의 결과가 포함된 state를 반환합니다.
    """
    state['answer'] = "단순 질문에 대한 답변입니다."
    return state


def web_search(state: AgentState) -> AgentState:
    """
    웹 검색이 필요한 질문을 처리하는 노드입니다.
    주택 공시가격, 공정시장가액비율 등 최신 정보를 외부에서 검색해옵니다.
    
    node 함수로 동작하므로 `state`를 인자로 받지만,
    고정된 기능(웹 검색)만 수행하기 때문에 내부적으로는 state를 활용하지 않습니다.
    
    Args:
        state (AgentState): 현재 에이전트의 state 객체입니다.
    
    Returns:
        AgentState: 'market_ratio' 또는 'answer' 키를 포함해 업데이트된 state를 반환합니다.
    """
    state['market_ratio'] = "웹 검색을 통해 가져온 공정시장가액비율 예시값"
    return state


def web_generate(state: AgentState) -> AgentState:
    """
    웹 검색 결과를 바탕으로 최종 답변을 생성하는 노드입니다.
    `web_search` 단계에서 가져온 정보를 LLM과 결합하여 자연스러운 답변을 만듭니다.
    
    Args:
        state (AgentState): 현재 에이전트의 state 객체입니다.
    
    Returns:
        AgentState: 'answer' 키에 최종 답변이 포함된 state를 반환합니다.
    """
    state['answer'] = f"웹 검색 기반 최종 답변입니다. (시장비율: {state.get('market_ratio', '없음')})"
    return state



In [9]:
graph_builder = StateGraph(AgentState)

# 노드 추가
graph_builder.add_node("income_tax", income_tax_agent)
graph_builder.add_node("real_estate_tax", real_estate_tax_agent)
graph_builder.add_node("basic_llm", basic_llm)
graph_builder.add_node("web_search", web_search)
graph_builder.add_node("web_generate", web_generate)

# 조건부 라우팅
graph_builder.add_conditional_edges(
    START,
    router,
    {
        "income_tax": "income_tax",
        "real_estate_tax": "real_estate_tax",
        "basic_llm": "basic_llm",
        "web_search": "web_search"
    }
)

# 웹 검색 후 generate
graph_builder.add_edge("web_search", "web_generate")

# 종료 지점 연결
graph_builder.add_edge("income_tax", END)
graph_builder.add_edge("real_estate_tax", END)
graph_builder.add_edge("basic_llm", END)
graph_builder.add_edge("web_generate", END)

# 그래프 완성
graph = graph_builder.compile()


In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))


RuntimeError: asyncio.run() cannot be called from a running event loop